In [1]:
#!pip install cassandra-driver

### Step1: Connection with cloud cassandra

In [18]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
        'secure_connect_bundle': 'F:\data science\ineuron\Databases\Cassandra\secure-connect-test1.zip'
}
auth_provider = PlainTextAuthProvider('xzwnPGxiZZZdvZHEGQeUjxEM', 'Iwr4DW-qeDFYgf,x1BtjHufBMgZE59r7pbHHeIvHPRx0p_h3OHLsssxDTU3_L_yK4DF0bouiFg0.2vxr.Z,1Dp3i5Kx2xZ4p7qqvINt,sJEco0fgMkRrZ7UaCf7-x2N2')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.0.6813


### 2. Show Available keyspaces

In [3]:
row = session.execute("SELECT * FROM system_schema.keyspaces")
for i in row:
    print(i[0])

data_endpoint_auth
abc
system_auth
system_schema
student
system
datastax_sla
system_traces


## CREATE TABLE

In [4]:
session.execute("USE STUDENT")
row = session.execute("CREATE TABLE STUDENTDETAILS(roll_no int PRIMARY KEY, fname text, lname text)")

In [5]:
#verification
row = session.execute("select * from studentdetails")
print(row.column_names)
print(row.one())
print(row.column_names,row.column_types)
for i in zip(row.column_names,row.column_types):
    print(i)

['roll_no', 'fname', 'lname']
None
['roll_no', 'fname', 'lname'] [<class 'cassandra.cqltypes.Int32Type'>, <class 'cassandra.cqltypes.VarcharType'>, <class 'cassandra.cqltypes.VarcharType'>]
('roll_no', <class 'cassandra.cqltypes.Int32Type'>)
('fname', <class 'cassandra.cqltypes.VarcharType'>)
('lname', <class 'cassandra.cqltypes.VarcharType'>)


In [6]:
session.execute("USE STUDENT")
row = session.execute("CREATE TABLE STUDENTDETAILS1(roll_no int PRIMARY KEY, fname text, lname text)")

## Drop Table

In [7]:
session.execute("DROP Table studentdetails1")

## Alter Table-add a column

In [8]:
session.execute("use student")
session.execute("ALTER TABLE studentdetails ADD contact_no int")

In [9]:
#verification
row = session.execute("select * from studentdetails")
print(row.column_names)

['roll_no', 'contact_no', 'fname', 'lname']


##  Alter Table-drop a column

In [10]:
session.execute("use student")
session.execute("ALTER TABLE studentdetails drop contact_no ")

In [11]:
#verification
row = session.execute("select * from studentdetails")
print(row.column_names)

['roll_no', 'fname', 'lname']


## Insert data

In [12]:
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(1,'joe','peterson')"
session.execute(query)

In [13]:
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(2,'mathew','forbes')"
session.execute(query)

In [14]:
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(3,'emily','bennett')"
session.execute(query)

In [15]:
#verification
row = session.execute("select * from studentdetails")
for i in row.all():
    print(i)   

Row(roll_no=1, fname='joe', lname='peterson')
Row(roll_no=2, fname='mathew', lname='forbes')
Row(roll_no=3, fname='emily', lname='bennett')


In [16]:
row = session.execute("select roll_no,fname from studentdetails")
row.all()

[Row(roll_no=1, fname='joe'),
 Row(roll_no=2, fname='mathew'),
 Row(roll_no=3, fname='emily')]

## Truncate Table

In [19]:
session.execute("Use student")
query = "Truncate studentdetails"
session.execute(query)

In [20]:
#verification
row = session.execute("select * from studentdetails")
row.all()

[]

In [21]:
query = "USE student"
session.execute(query)
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(1,'joe','peterson')"
session.execute(query)
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(2,'mathew','forbes')"
session.execute(query)
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(3,'emily','bennett')"
session.execute(query)

## Update 

In [22]:
query = "UPDATE studentdetails SET lname = 'clarks' where roll_no=2"
session.execute(query)

In [23]:
#verification
query = "Select * from studentdetails"
row= session.execute(query)
row.all()

[Row(roll_no=1, fname='joe', lname='peterson'),
 Row(roll_no=2, fname='mathew', lname='clarks'),
 Row(roll_no=3, fname='emily', lname='bennett')]

# Batch
- BATCH is used to execute multiple modification statements (insert, update, delete) simultaneously. It is very useful when you have to update some column as well as delete some of the existing.
-  only create, update, and delete rows with a batch query, attempting to read rows out of the database with a batch query will fail.

In [24]:
session.execute("USE STUDENT")
query = "BEGIN BATCH INSERT INTO studentdetails(roll_no,fname,lname) values(4,'joeye','john');" + " UPDATE studentdetails SET fname = 'joey' where roll_no = 1;" + "DELETE FROM studentdetails WHERE roll_no = 2;" +   "APPLY BATCH;"
session.execute(query)

In [25]:
query = "SELECT * from studentdetails"
row= session.execute(query)
row.all()

[Row(roll_no=1, fname='joey', lname='peterson'),
 Row(roll_no=4, fname='joeye', lname='john'),
 Row(roll_no=3, fname='emily', lname='bennett')]

In [26]:
query = "BEGIN BATCH INSERT INTO studentdetails(roll_no,fname,lname)values(5,'Paul', 'Walker')" + "INSERT INTO studentdetails(roll_no,fname,lname)values(6, 'Ricky', 'Ponting')" + "INSERT INTO studentdetails(roll_no,fname,lname)values(7,'asim','akram')" + "INSERT INTO studentdetails(roll_no,fname,lname)values(8,'sunil', 'gawaskar')" + "APPLY BATCH;"
session.execute(query)

In [27]:
query = "SELECT * from studentdetails"
row= session.execute(query)
row.all()

[Row(roll_no=5, fname='Paul', lname='Walker'),
 Row(roll_no=1, fname='joey', lname='peterson'),
 Row(roll_no=8, fname='sunil', lname='gawaskar'),
 Row(roll_no=4, fname='joeye', lname='john'),
 Row(roll_no=7, fname='asim', lname='akram'),
 Row(roll_no=6, fname='Ricky', lname='Ponting'),
 Row(roll_no=3, fname='emily', lname='bennett')]

INSERT: user input and for loop

In [28]:
query = "INSERT INTO studentdetails(roll_no,fname,lname)values(%s,%s,%s);" 
for i in range(10):
    a1=int(input("Enter roll_no here:"))
    b1=input("Enter firstname here:")
    c1=input("Enter Lastname here: ")
    data= (a1,b1,c1)

    session.execute(query,data)

Enter roll_no here:11
Enter firstname here:Roger
Enter Lastname here: federer
Enter roll_no here:12
Enter firstname here:Andre
Enter Lastname here: agassi
Enter roll_no here:13
Enter firstname here:Anurag
Enter Lastname here: kashyap
Enter roll_no here:14
Enter firstname here:Karan
Enter Lastname here: Johar
Enter roll_no here:15
Enter firstname here:Julie
Enter Lastname here: Plec
Enter roll_no here:16
Enter firstname here:Graham
Enter Lastname here: Bell
Enter roll_no here:17
Enter firstname here:Kat
Enter Lastname here: Graham
Enter roll_no here:18
Enter firstname here:Sophie
Enter Lastname here: Turner
Enter roll_no here:19
Enter firstname here:Arya
Enter Lastname here: Stark
Enter roll_no here:20
Enter firstname here:James
Enter Lastname here: Lannister


In [30]:
def insertcommand(rollno,firstname,lastname):
    rollno = int(rollno)
    try:
        data= (rollno,firstname,lastname)
        query = "INSERT INTO studentdetails(roll_no,fname,lname)values(%s,%s,%s);" 
        session.execute(query,data)
    except:
        print("Cant be inserted")
    
insertcommand(9,'rishabh','pant')
    

In [31]:
query = "SELECT * from studentdetails"
row= session.execute(query)
row.all()

[Row(roll_no=5, fname='Paul', lname='Walker'),
 Row(roll_no=16, fname='Graham', lname='Bell'),
 Row(roll_no=13, fname='Anurag', lname='kashyap'),
 Row(roll_no=11, fname='Roger', lname='federer'),
 Row(roll_no=1, fname='joey', lname='peterson'),
 Row(roll_no=19, fname='Arya', lname='Stark'),
 Row(roll_no=8, fname='sunil', lname='gawaskar'),
 Row(roll_no=4, fname='joeye', lname='john'),
 Row(roll_no=18, fname='Sophie', lname='Turner'),
 Row(roll_no=15, fname='Julie', lname='Plec'),
 Row(roll_no=20, fname='James', lname='Lannister'),
 Row(roll_no=7, fname='asim', lname='akram'),
 Row(roll_no=6, fname='Ricky', lname='Ponting'),
 Row(roll_no=9, fname='rishabh', lname='pant'),
 Row(roll_no=14, fname='Karan', lname='Johar'),
 Row(roll_no=17, fname='Kat', lname='Graham'),
 Row(roll_no=12, fname='Andre', lname='agassi'),
 Row(roll_no=3, fname='emily', lname='bennett')]

In [32]:
query = "ALTER table studentdetails add TOTAL_Marks int"
session.execute(query)

In [33]:
#verification
query = "SELECT * from studentdetails"
row= session.execute(query)
row.column_names

['roll_no', 'fname', 'lname', 'total_marks']

In [34]:
query = "SELECT * from studentdetails"
row= session.execute(query)

query = "UPDATE studentdetails SET total_marks = %s WHERE roll_no = %s"
for i in row.all():  
    print("Roll_no",i[0], "and student name is ",i[1])
    total = int(input("Enter marks for the student"))
    data = (total,i[0])
    session.execute(query,data)
    

Roll_no 5 and student name is  Paul
Enter marks for the student76
Roll_no 16 and student name is  Graham
Enter marks for the student56
Roll_no 13 and student name is  Anurag
Enter marks for the student65
Roll_no 11 and student name is  Roger
Enter marks for the student78
Roll_no 1 and student name is  joey
Enter marks for the student56
Roll_no 19 and student name is  Arya
Enter marks for the student98
Roll_no 8 and student name is  sunil
Enter marks for the student76
Roll_no 4 and student name is  joeye
Enter marks for the student45
Roll_no 18 and student name is  Sophie
Enter marks for the student98
Roll_no 15 and student name is  Julie
Enter marks for the student100
Roll_no 20 and student name is  James
Enter marks for the student23
Roll_no 7 and student name is  asim
Enter marks for the student34
Roll_no 6 and student name is  Ricky
Enter marks for the student78
Roll_no 9 and student name is  rishabh
Enter marks for the student89
Roll_no 14 and student name is  Karan
Enter marks for

In [35]:
#verification
query = "SELECT * from studentdetails"
row= session.execute(query)
row.all()

[Row(roll_no=5, fname='Paul', lname='Walker', total_marks=76),
 Row(roll_no=16, fname='Graham', lname='Bell', total_marks=56),
 Row(roll_no=13, fname='Anurag', lname='kashyap', total_marks=65),
 Row(roll_no=11, fname='Roger', lname='federer', total_marks=78),
 Row(roll_no=1, fname='joey', lname='peterson', total_marks=56),
 Row(roll_no=19, fname='Arya', lname='Stark', total_marks=98),
 Row(roll_no=8, fname='sunil', lname='gawaskar', total_marks=76),
 Row(roll_no=4, fname='joeye', lname='john', total_marks=45),
 Row(roll_no=18, fname='Sophie', lname='Turner', total_marks=98),
 Row(roll_no=15, fname='Julie', lname='Plec', total_marks=100),
 Row(roll_no=20, fname='James', lname='Lannister', total_marks=23),
 Row(roll_no=7, fname='asim', lname='akram', total_marks=34),
 Row(roll_no=6, fname='Ricky', lname='Ponting', total_marks=78),
 Row(roll_no=9, fname='rishabh', lname='pant', total_marks=89),
 Row(roll_no=14, fname='Karan', lname='Johar', total_marks=76),
 Row(roll_no=17, fname='Kat', l

## Aggregation
- sum      = sum()
- average  = avg()
- minimum  = min()
- maximum  = max()
- count    = count()

In [36]:
query = "Select sum(total_marks) from studentdetails"
row = session.execute(query)
row.one()

Row(system_sum_total_marks=1333)

In [37]:
query = "Select avg(total_marks) from studentdetails"
row = session.execute(query)
row.one()

Row(system_avg_total_marks=74)

In [38]:
query = "Select min(total_marks) from studentdetails"
row = session.execute(query)
row.one()

Row(system_min_total_marks=23)

In [39]:
query = "Select max(total_marks) from studentdetails"
row = session.execute(query)
row.one()

Row(system_max_total_marks=100)

In [40]:
query = "Select count(total_marks) from studentdetails"
row = session.execute(query)
row.one()

Row(system_count_total_marks=18)

In [41]:
query = "Select count(*) from studentdetails"
row = session.execute(query)
row.one()

Row(count=18)

## Allow Filtering


In [42]:
query = "Select * from studentdetails Where total_marks>70 ALLOW FILTERING"
row = session.execute(query)
row.all()

[Row(roll_no=5, fname='Paul', lname='Walker', total_marks=76),
 Row(roll_no=11, fname='Roger', lname='federer', total_marks=78),
 Row(roll_no=19, fname='Arya', lname='Stark', total_marks=98),
 Row(roll_no=8, fname='sunil', lname='gawaskar', total_marks=76),
 Row(roll_no=18, fname='Sophie', lname='Turner', total_marks=98),
 Row(roll_no=15, fname='Julie', lname='Plec', total_marks=100),
 Row(roll_no=6, fname='Ricky', lname='Ponting', total_marks=78),
 Row(roll_no=9, fname='rishabh', lname='pant', total_marks=89),
 Row(roll_no=14, fname='Karan', lname='Johar', total_marks=76),
 Row(roll_no=17, fname='Kat', lname='Graham', total_marks=98),
 Row(roll_no=12, fname='Andre', lname='agassi', total_marks=99),
 Row(roll_no=3, fname='emily', lname='bennett', total_marks=88)]

In [43]:
query = "Select * from studentdetails Where total_marks>70 and total_marks<90 ALLOW FILTERING"
row = session.execute(query)
row.all()

[Row(roll_no=5, fname='Paul', lname='Walker', total_marks=76),
 Row(roll_no=11, fname='Roger', lname='federer', total_marks=78),
 Row(roll_no=8, fname='sunil', lname='gawaskar', total_marks=76),
 Row(roll_no=6, fname='Ricky', lname='Ponting', total_marks=78),
 Row(roll_no=9, fname='rishabh', lname='pant', total_marks=89),
 Row(roll_no=14, fname='Karan', lname='Johar', total_marks=76),
 Row(roll_no=3, fname='emily', lname='bennett', total_marks=88)]

## order by
- The Partition Key is responsible for data distribution across your nodes. The Clustering Key is responsible for data sorting within the partition. The Primary Key is equivalent to the Partition Key in a single-field-key table
- The partition key must be defined in the WHERE clause and the ORDER BY clause defines the clustering column to use for ordering.

In [51]:
query = "Create table order_by_totalmarks(roll_no int, fname text, lname text,total_marks int,category text ,PRIMARY KEY(category,total_marks))"
row = session.execute(query)


In [55]:
query = "SELECT * from studentdetails"
row= session.execute(query)
l = []
for i in row:
    l.append(i)
print(len(l))

18


In [56]:
from cassandra.query import SimpleStatement, BatchStatement
batch = BatchStatement()
for i in l:
    batch.add(SimpleStatement("INSERT INTO order_by_totalmarks(roll_no,fname, lname,total_marks,category)values(%s,%s,%s,%s,%s)"),(i[0],i[1],i[2],i[3],"ordered"))
session.execute(batch)

In [57]:
query = "select * from order_by_totalmarks where category = 'ordered' order by total_marks"
row = session.execute(query)
row.all()

[Row(category='ordered', total_marks=23, fname='James', lname='Lannister', roll_no=20),
 Row(category='ordered', total_marks=34, fname='asim', lname='akram', roll_no=7),
 Row(category='ordered', total_marks=45, fname='joeye', lname='john', roll_no=4),
 Row(category='ordered', total_marks=56, fname='joey', lname='peterson', roll_no=16),
 Row(category='ordered', total_marks=65, fname='Anurag', lname='kashyap', roll_no=13),
 Row(category='ordered', total_marks=76, fname='sunil', lname='gawaskar', roll_no=14),
 Row(category='ordered', total_marks=78, fname='Roger', lname='federer', roll_no=11),
 Row(category='ordered', total_marks=88, fname='emily', lname='bennett', roll_no=3),
 Row(category='ordered', total_marks=89, fname='rishabh', lname='pant', roll_no=9),
 Row(category='ordered', total_marks=98, fname='Sophie', lname='Turner', roll_no=19),
 Row(category='ordered', total_marks=99, fname='Andre', lname='agassi', roll_no=12),
 Row(category='ordered', total_marks=100, fname='Julie', lname

##  AS and LIMIT

In [58]:
query = "select fname AS TOP5 , total_marks from order_by_totalmarks where category = 'ordered' order by total_marks DESC LIMIT 5"
row = session.execute(query)
row.all()

[Row(top5='Julie', total_marks=100),
 Row(top5='Andre', total_marks=99),
 Row(top5='Sophie', total_marks=98),
 Row(top5='rishabh', total_marks=89),
 Row(top5='emily', total_marks=88)]

## Create Index
- An index provides a means to access data in Cassandra using attributes other than the partition key. 
- The benefit is fast, efficient lookup of data matching a given condition. The index indexes column values in a separate, hidden table from the one that contains the values being indexed. 
- Cassandra has a number of techniques for guarding against the undesirable scenario where data might be incorrectly retrieved during a query involving indexes on the basis of stale values in the index.
- Indexes can be used for collections, collection columns, and any other columns except counter columns and static columns.

### When NOT TO use INDEX
- column with high cardinality(large number of unique values) for smaller set of result
- column with extremely low cardinality 

In [59]:
query = "Create Index firstname on studentdetails(fname)"
session.execute(query)

In [60]:
#verification
query = "Create Index firstname on studentdetails(fname)"
session.execute(query)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Index 'firstname' already exists"

### DROP INDEX

In [61]:
query = "DROP INDEX IF EXISTS student.fname"
session.execute(query)


In [62]:

#verification
query = "DROP INDEX student.firstname"
session.execute(query)